In [1]:
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from peft import AutoPeftModelForCausalLM

/home/bhx5gh/Documents/NLP/NLP_Final_Political_Bias_Shifts/.venv/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
trained_model_path = '/home/bhx5gh/Documents/NLP/NLP_Final_Political_Bias_Shifts/fine_tuned_llms/mistralai_Mistral-7B-v0_1/checkpoints/2024-12-14 13-27-46/checkpoint-9560'
base_model_path = "/home/bhx5gh/Documents/NLP/NLP_Final_Political_Bias_Shifts/src/Local Models/mistralai_Mistral-7B-v0_1"

base_model = AutoModelForCausalLM.from_pretrained(base_model_path, torch_dtype=torch.float16, device_map = "auto")

trained_model =  AutoPeftModelForCausalLM.from_pretrained(trained_model_path, torch_dtype=torch.float16)
trained_model = trained_model.to("cuda")
trained_model.eval()
tokenizer = AutoTokenizer.from_pretrained(base_model_path)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

base_pipe = pipeline(
    "text-generation", 
    model=base_model, 
    tokenizer = tokenizer,
    torch_dtype=torch.float16, 
    device_map="auto",
    max_length = 100
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:13<00:00,  6.67s/it]
Device set to use cuda:0


In [3]:
input_text = "My thoughts on the economy are "
input_text = input_text.encode('utf-8').decode('utf-8')

print(f'Baseline:\n {base_pipe(input_text)}\n\n')

input_tokens = tokenizer(input_text, return_tensors="pt")
outputs = trained_model.generate(input_ids=input_tokens["input_ids"].to("cuda"), max_new_tokens=100)
outputs_decoded = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]

print(f'Trained model:\n{outputs_decoded}\n\n')

token_ids = outputs.detach().cpu().numpy()[0]  # Convert to NumPy array and select the first sequence

# Print the token IDs
print("Generated token IDs:", token_ids)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Baseline:
 [{'generated_text': 'My thoughts on the economy are 100% in line with the following article.\n\n> The U.S. economy is in a recession.\n>\n> That’s the conclusion of a growing number of economists, who say the economy is in a recession that began in December.\n>\n> The National Bureau of Economic Research, the official arbiter of recessions, has yet to make a call. But the NBER’s definition of'}]


Trained model:
My thoughts on the economy are 1. The economy is not doing well. 2. The economy is not doing well because of the policies of the Biden administration. 3. The economy is not doing well because of the policies of the Biden administration and the policies of the Trump administration. 4. The economy is not doing well because of the policies of the Biden administration and the policies of the Trump administration and the policies of the Obama administration. 5. The economy is not doing well because of the policies of the Biden


Generated token IDs: [    1  1984  7403   3

In [4]:
input_text = "What are your thoughts on the Economy?"
input_tokens = tokenizer(input_text, return_tensors="pt")
outputs = tokenizer.batch_decode(input_tokens, skip_special_tokens=False)[0]

TypeError: argument 'ids': Can't extract `str` to `Vec`